In [1]:
!pip install gluonnlp==0.10.0 pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.2

     |████████████████████████████████| 344 kB 14.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595725 sha256=e4d01db52a8758ecce5bc30defbb8b568a3ea15f45c9d4023bf34ec921573e35
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 46.9 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 1.1 MB 12.1 MB/s 
     |████████████████████████████████| 2.5 MB 15.5 MB/s 
     |████████████████████████████████| 895 kB 78.0 MB/s 
     |████████████████████████████████| 3.3 MB 70.1 MB/s 
ERROR: Could not find a version that satisfies the requirement torch==1.8.2 (from versions: 0.4.1, 0.4.1.post2, 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1,

In [2]:
pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-7slk47vf/kobert-tokenizer_558a1733523e479bb09bedb782a033dd
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-7slk47vf/kobert-tokenizer_558a1733523e479bb09bedb782a033dd
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4682 sha256=8c6a7847993fc8a7885cccd9ed4cf6271b1497cff5a549d24b4fe4e159764df6
  Stored in directory: /tmp/pip-ephem-wheel-cache-q233bako/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model
# from kobert.pytorch_kobert import get_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from google.colab import drive
import sentencepiece
import transformers
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


# 에러나서 다른 방법 사용

In [4]:
print(torch.__version__) #1.8.1
print(transformers.__version__) #4.8.2

1.10.0+cu111
4.8.2


In [5]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-26n56wrk
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-26n56wrk
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=589d5cd4edf7e7564373f53c5ecb8c4004bd00206a629d49d8b02a709957e5ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-8xu4hqop/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [6]:
from kobert_tokenizer import KoBERTTokenizer
from kobert.pytorch_kobert import get_kobert_model
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

In [7]:
#띄어쓰기에 강건한 모델
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})

In [8]:
import torch
from transformers import BertModel
# model = BertModel.from_pretrained('skt/kobert-base-v1')
# text = "한국어 모델을 공유합니다."
# inputs = tokenizer.batch_encode_plus([text])
# out = model(input_ids = torch.tensor(inputs['input_ids']),attention_mask = torch.tensor(inputs['attention_mask']))
# out.pooler_output.shape

In [9]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

# 데이터 전처리

In [ ]:
saturi = pd.read_csv('/content/drive/MyDrive/자연어처리 과제/all_표준어.csv')

In [ ]:
import re
data_list = []
for q, ind in zip(saturi['text'], list(saturi.index))  :
    data = []
    q = str(q)
    # q = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', q)
    # q = re.sub('[A-Za-z]+_[A-Za-z]*','',q)
    # q = q.strip()
    if len(q)>=30:
        data.append(q)
        data.append(str(ind))

        data_list.append(data)

In [ ]:
import random

data_list = random.sample(data_list, 50000)

In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

120000
30000


In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])
np.unique(saturi['label'])

['아침저녁만 매경   점심은 안 매기고   그날 열두 명도 내고 열세 명도 되고', '3']
['그런 애들이 오히려 더 더 싹퉁이 없고 쫌   개념이 없어', '1']
['요즘 안 먹어 근데 얼굴살이 쪽 빠졌지 아네', '3']
['태아의 성별은 수정 때 하마   결정돼', '0']
['참기를을 발라 쇵편  을 뺀질뺀질  하게 해라', '0']
['칠억에 팔았어 칠억 이천에 팔아서 그해 구십 프로 팔십 프로 구십 프로 시가로 세금을 때려준다는 거여', '4']
['야이  도 이제 자기 죄책감 덜어볼라구', '3']


array([0, 1, 2, 3, 4])

In [11]:
# Setting parameters
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 2  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#토큰화 및 모델 돌리기

In [ ]:
#토큰화
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok,vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok,vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
device = torch.device("cpu")
tok=tokenizer.tokenize

#BERT 모델 불러오기
# model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model = torch.load('/content/drive/MyDrive/자연어처리 과제/model/표준어_감정분석_다른애들로_epoch10.h5')
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

# t_total = len(train_dataloader) * num_epochs
# warmup_step = int(t_total * warmup_ratio)

# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# #정확도 측정을 위한 함수 정의
# def calc_accuracy(X,Y):
#     max_vals, max_indices = torch.max(X, 1)
#     train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
#     return train_acc
    
# train_dataloader

# 모델 학습

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/938 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.5526227951049805 train acc 0.2890625
epoch 1 batch id 201 loss 1.0204046964645386 train acc 0.46606809701492535
epoch 1 batch id 401 loss 0.7750181555747986 train acc 0.5470503428927681
epoch 1 batch id 601 loss 0.7537219524383545 train acc 0.5949849209650583
epoch 1 batch id 801 loss 0.5258233547210693 train acc 0.6252243289637952


In [ ]:
model.save('/content/drive/MyDrive/자연어처리 과제/model/saturi_지역분류.h5')

In [ ]:
torch.save(model, '/content/drive/MyDrive/자연어처리 과제/model/사투리_지역분류_epoch2.h5')

In [ ]:
data_list

[['그리 마스크   조금 불편한 점도 있지만 좋은 점이 너무 많아', '2261089'],
 ['드러누워 있으면 할아버지가 같이 식사를 못 하시는 거라', '2227646'],
 ['그래 가지고   조금 그런 어 아기자기하고 쪼끔 사람들이 덜 붐비는', '7808782'],
 ['백건우근데 안에 분위기 자체 거의 그냥   어지간한 모텔급으로 그냥 침대 있고 근데 전기장판만 없는 거지 침대에', '7611109'],
 ['셋방이 미니 일 층이에요   그러니까 이쪽에서 들어가면 일 층이고', '8059432'],
 ['자기가 꼭 그렇게 하고 다니고 아무리 막 임신을 하고 다녀도   자기 스타일 딱 지키더라고 사무실을 갈 때도', '7506373'],
 ['성격은 이게 하고 싶어도 참는 거야 내가 내가 잠 잘 자고 내가   편하려고', '7397596'],
 ['xxxx 큰아이   작은아이가 지금 인천에   같이   사는데', '4387172'],
 ['한 오천만 원 정도 했던 것   같아 오천만 원 오천도 뭐', '6346270'],
 ['야가 얘가   아니고   쟤가   쟤고    굉장히 막 헷갈리게', '2684355'],
 ['그러니까 나는 근데 실제 성격이랑 다른 거에서 진짜 연기면 연기 잘한다고 느껸', '5879555'],
 ['그래   눈을   바짝 갖다 대고   조준구명를 들여다 봐야지', '9314'],
 ['걱정이야 괜히  걱정하고 걱정만 하고 있어서 잘 하지도 못하면서', '3160695'],
 ['왜 그냥 사람인 게 아 맞아   이 그러면 사람이면 언제 죽을지 모르지', '7250550'],
 ['나는 초반에   그렇게 못해났거든 그래서 막 검색해 보고   그렇게했었는데', '4049935'],
 ['이예진나는 막 대전 밖을 많이 벗어나는 편이   아니어서', '7590861'],
 ['굉장히 괴롭고 막   너도 음악 좋아하잖아 어 음악 좋아하니까 그 악기에도 관심이 많고 막   그렇지   않니 근데',
  '1968616'],
 ['네 이

In [ ]:
import pickle
with open("/content/drive/MyDrive/자연어처리 과제/model/5만개.txt", "wb") as fp:   #Pickling
    pickle.dump(data_list, fp)

In [13]:
import pickle

with open("/content/drive/MyDrive/자연어처리 과제/model/5만개.txt", "rb") as fp:   # Unpickling
    data_list = pickle.load(fp)

# 검증

In [15]:
#토큰화
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            test_eval.append(np.argmax(logits))
        return test_eval[0]

In [16]:
counter = 25000
predict_label = []
for i in data_list[25000:] :
    print(counter)
    sentence = i[0]
    predict_label.append(predict(sentence))
    counter += 1

25000


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
45000
45001
45002
45003
45004
45005
45006
45007
45008
45009
45010
45011
45012
45013
45014
45015
45016
45017
45018
45019
45020
45021
45022
45023
45024
45025
45026
45027
45028
45029
45030
45031
45032
45033
45034
45035
45036
45037
45038
45039
45040
45041
45042
45043
45044
45045
45046
45047
45048
45049
45050
45051
45052
45053
45054
45055
45056
45057
45058
45059
45060
45061
45062
45063
45064
45065
45066
45067
45068
45069
45070
45071
45072
45073
45074
45075
45076
45077
45078
45079
45080
45081
45082
45083
45084
45085
45086
45087
45088
45089
45090
45091
45092
45093
45094
45095
45096
45097
45098
45099
45100
45101
45102
45103
45104
45105
45106
45107
45108
45109
45110
45111
45112
45113
45114
45115
45116
45117
45118
45119
45120
45121
45122
45123
45124
45125
45126
45127
45128
45129
45130
45131
45132
45133
45134
45135
45136
45137
45138
45139
45140
45141
45142
45143
45144
45145
45146
45147
45148
45149
45150
45151
45152
45153
45154
45155
45156
45157
45158
45159
4516

In [17]:
import pickle
with open("/content/drive/MyDrive/자연어처리 과제/model/25000_50000.txt", "wb") as fp:   #Pickling
    pickle.dump(predict_label, fp)

In [1]:
!pip install torch==1.6.0 torchvision==0.7.0 torchaudio==0.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install torchtext==0.9.1

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |████████████████████████████████| 748.8 MB 17 kB/s 
     |████████████████████████████████| 5.9 MB 25.9 MB/s 
     |████████████████████████████████| 1.9 MB 43.4 MB/s 
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==1.6.0, torchaudio==0.8.1 and torchvision==0.7.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==1.6.0
    torchvision 0.7.0 depends on torch==1.6.0
    torchaudio 0.8.1 depends on torch==1.8.1

T

In [2]:
!pip install git+https://github.com/kakaobrain/pororo.git

  Cloning https://github.com/kakaobrain/pororo.git to /tmp/pip-req-build-obddpil6
  Running command git clone -q https://github.com/kakaobrain/pororo.git /tmp/pip-req-build-obddpil6
  Using cached torch-1.6.0-cp37-cp37m-manylinux1_x86_64.whl (748.8 MB)
  Using cached torchvision-0.7.0-cp37-cp37m-manylinux1_x86_64.whl (5.9 MB)
     |████████████████████████████████| 1.7 MB 36.5 MB/s 
     |████████████████████████████████| 3.1 MB 46.4 MB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 1.5 MB 30.2 MB/s 
     |████████████████████████████████| 3.1 MB 19.7 MB/s 
     |████████████████████████████████| 468 kB 47.6 MB/s 
     |████████████████████████████████| 1.2 MB 48.8 MB/s 
     |████████████████████████████████| 42.4 MB 55.9 MB/s 
     |████████████████████████████████| 90 kB 9.6 MB/s 
     |████████████████████████████████| 145 kB 43.9 MB/s 
     |████████████████████████████████| 749 kB 43.7 MB/s 
     |███████████████████████████████

In [3]:
import torch
# import timm
print(torch.__version__)
# print(timm.__version__)

1.6.0


In [4]:
from pororo import Pororo
sa = Pororo(task="sentiment", model="brainbert.base.ko.nsmc", lang="ko")

In [5]:
# import torch
# from torch import nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import gluonnlp as nlp
# import numpy as np
# from tqdm import tqdm, tqdm_notebook
# #kobert
# # from kobert.utils import get_tokenizer
# # from kobert.pytorch_kobert import get_pytorch_kobert_model
# # from kobert.pytorch_kobert import get_kobert_model

# #transformers
# from transformers import AdamW
# from transformers.optimization import get_cosine_schedule_with_warmup
from google.colab import drive
# import sentencepiece
# import transformers
# import torch
# from torch import nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import gluonnlp as nlp
# import numpy as np
# from tqdm import tqdm, tqdm_notebook
# import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pickle

with open("/content/drive/MyDrive/자연어처리 과제/model/5만개.txt", "rb") as fp:   # Unpickling
    data_list = pickle.load(fp)

In [7]:
counter = 25000
predict_label = []
for i in data_list[25000:] :
    print(counter)
    sentence = i[0]
    predict_label.append(sa(sentence))
    counter += 1

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
45000
45001
45002
45003
45004
45005
45006
45007
45008
45009
45010
45011
45012
45013
45014
45015
45016
45017
45018
45019
45020
45021
45022
45023
45024
45025
45026
45027
45028
45029
45030
45031
45032
45033
45034
45035
45036
45037
45038
45039
45040
45041
45042
45043
45044
45045
45046
45047
45048
45049
45050
45051
45052
45053
45054
45055
45056
45057
45058
45059
45060
45061
45062
45063
45064
45065
45066
45067
45068
45069
45070
45071
45072
45073
45074
45075
45076
45077
45078
45079
45080
45081
45082
45083
45084
45085
45086
45087
45088
45089
45090
45091
45092
45093
45094
45095
45096
45097
45098
45099
45100
45101
45102
45103
45104
45105
45106
45107
45108
45109
45110
45111
45112
45113
45114
45115
45116
45117
45118
45119
45120
45121
45122
45123
45124
45125
45126
45127
45128
45129
45130
45131
45132
45133
45134
45135
45136
45137
45138
45139
45140
45141
45142
45143
45144
45145
45146
45147
45148
45149
45150
45151
45152
45153
45154
45155
45156
45157
45158
45159
4516

In [8]:
import pickle
with open("/content/drive/MyDrive/자연어처리 과제/model/25000_50000_긍부정.txt", "wb") as fp:   #Pickling
    pickle.dump(predict_label, fp)